<a href="https://colab.research.google.com/github/iam-fern/Research/blob/main/sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from numpy.random import normal
import statistics

In [2]:
# numpy.random.normal(loc=0.0, scale=1.0, size=None)
# generate sample of 200 values that follow a normal distribution 
# data = normal(loc=0, scale=1, size=200)

data = pd.DataFrame({"zt1" : normal(0, 1, 20000),
                     "zt2" : normal(0, 1, 20000),
                     "zt3" : normal(0, 1, 20000)})
data

,zt1,zt2,zt3
0,-0.169945,0.577454,-1.287822
1,0.575656,0.035264,-0.047030
2,1.336883,0.466323,0.129764
3,0.531904,-0.797853,-0.689654
4,0.286291,0.176691,2.640079
...,...,...,...
19995,1.665243,1.010604,0.006001
19996,1.789026,-1.066644,-0.103790
19997,1.352673,-0.412280,0.922039
19998,-0.566153,-0.007469,0.604440


In [3]:
p = 0

In [4]:
Zt1 = data['zt1']
Zt2 = data['zt2']
Zt3 = data['zt3']

In [5]:
xt1 = (1-(p**2))**(1/2)*Zt1
xt2 = (1-(p**2))**(1/2)*Zt2
xt3 = (1-(p**2))**(1/2)*Zt3

In [6]:
new_data = pd.DataFrame({'Xt1' : data['zt1'],
                         'Xt2' : data['zt2'],
                         'Xt3' : data['zt3'],
                         'e'   : normal(0, xt2**4, 20000)})
new_data

,Xt1,Xt2,Xt3,e
0,-0.169945,0.577454,-1.287822,2.436656e-02
1,0.575656,0.035264,-0.047030,3.227903e-06
2,1.336883,0.466323,0.129764,-2.781245e-02
3,0.531904,-0.797853,-0.689654,-1.002437e+00
4,0.286291,0.176691,2.640079,7.894962e-04
...,...,...,...,...
19995,1.665243,1.010604,0.006001,-4.255541e-01
19996,1.789026,-1.066644,-0.103790,-4.066139e-01
19997,1.352673,-0.412280,0.922039,-6.235365e-03
19998,-0.566153,-0.007469,0.604440,-1.437503e-09


In [8]:
table_data = pd.DataFrame({'Xt1' : data['zt1'],
                           'Xt2' : data['zt2'],
                           'Xt3' : data['zt3'],
                           'e'   : new_data['e']})
table_data['Y'] = 4+0.4*table_data['Xt1']+1.5*table_data['Xt2']+3.6*table_data['Xt3']+table_data['e']
table_data['Y_hat'] = 4+0.4*table_data['Xt1']+1.5*table_data['Xt2']+3.6*table_data['Xt3']
table_data['e_hat']=table_data['Y']-table_data['Y_hat']
table_data

,Xt1,Xt2,Xt3,e,Y,Y_hat,e_hat
0,-0.169945,0.577454,-1.287822,2.436656e-02,0.186411,0.162044,2.436656e-02
1,0.575656,0.035264,-0.047030,3.227903e-06,4.113853,4.113850,3.227903e-06
2,1.336883,0.466323,0.129764,-2.781245e-02,5.673576,5.701388,-2.781245e-02
3,0.531904,-0.797853,-0.689654,-1.002437e+00,-0.469210,0.533227,-1.002437e+00
4,0.286291,0.176691,2.640079,7.894962e-04,13.884626,13.883836,7.894962e-04
...,...,...,...,...,...,...,...
19995,1.665243,1.010604,0.006001,-4.255541e-01,5.778054,6.203608,-4.255541e-01
19996,1.789026,-1.066644,-0.103790,-4.066139e-01,2.335389,2.742003,-4.066139e-01
19997,1.352673,-0.412280,0.922039,-6.235365e-03,7.235753,7.241989,-6.235365e-03
19998,-0.566153,-0.007469,0.604440,-1.437503e-09,5.938321,5.938321,-1.437503e-09
